# Naive Bayes


En este notebook se van a estar trabajando los modelos basados en Naive Bayes. Estos modelos son relativamente sencillos. ....

## Configuraciones iniciales

Cargamos las bibliotecas correspondientes a este modelo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

from preprocessing import prepararSetDeDatos
from preprocessing import categoricalNBPreprocessing
from preprocessing import multinomialNBPreprocessing
from preprocessing import gaussianNBPreprocessing

## Carga y preparacion del set de datos

Cargamos ambos set de datos, y se los pasamos a la funcion que realiza el armado hecho para el TP1.

In [ ]:
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')

In [ ]:
X,y = prepararSetDeDatos(X,y)

## Funciones auxiliares

In [ ]:
def obtenerMejorAlpha(modelo,datosPreprocesados):
    mejor_valor = 0
    mejor_alpha = None
    y_array=np.array(y)
    for valor_alpha in [0.001,0.01,0.1,0.3,0.5,0.7,1,2,3,10]:
        kf = StratifiedKFold(n_splits=8)
        metricas = []
        for fold_idx, (train_index, test_index) in enumerate(kf.split(datosPreprocesados, y_array)):
            modeloNB = modelo(alpha=valor_alpha)
            modeloNB.fit(datosPreprocesados[train_index], y_array[train_index].ravel())
            predicciones = modeloNB.predict(datosPreprocesados[test_index])
            score_obtenida = roc_auc_score(y_array[test_index],predicciones)
            metricas.append(score_obtenida)

        if np.mean(metricas) >= mejor_valor:
            mejor_valor = np.mean(metricas)
            mejor_alpha = valor_alpha
            
    return mejor_valor, mejor_alpha

In [ ]:
def graficarAUCROC(tipo,modeloNB,X_test,X_train,y_test,y_train):
    
    fpr_nb_test,tpr_nb_test,thresholds_nb_test = roc_curve(y_test,modeloNB.predict_proba(X_test)[:,1])
    fpr_nb_train,tpr_nb_train,thresholds_nb_train = roc_curve(y_train,modeloNB.predict_proba(X_train)[:,1])

    zero_test = np.argmin(np.abs(thresholds_nb_test))
    zero_train = np.argmin(np.abs(thresholds_nb_train))

    plt.plot(fpr_nb_train,tpr_nb_train,label="ROC Curve "+tipo+" NB Train")
    plt.plot(fpr_nb_test,tpr_nb_test,label="ROC Curve  "+tipo+" NB Test")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.plot(fpr_nb_test[zero_test],tpr_nb_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)
    plt.plot(fpr_nb_train[zero_train],tpr_nb_train[zero_train],'x',markersize=10,label="threshold zero train",fillstyle="none",c="k",mew=2)

    plt.legend(loc=4)
    plt.show()

## CategoricalNB

In [ ]:
X_categoricalNB = categoricalNBPreprocessing(X)

In [ ]:
#mejor_valor, mejor_alpha = obtenerMejorAlpha(CategoricalNB,X_categoricalNB) # Tira el index out of bounds (index 6 is out of bounds for axis 1 with size 6)
mejor_valor = 1
mejor_alpha = 1

In [ ]:
print(mejor_alpha)
print(mejor_valor)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_categoricalNB, y, test_size=0.25, random_state=0)

In [ ]:
categoricalNB = CategoricalNB(alpha=mejor_alpha)

In [ ]:
categoricalNB.fit(X_train, y_train)

Realizamos ahora las predicciones correspondientes con la parte de entrenamiento y la de pruebas, una vez hecho eso, mostramos las metricas obtenidas en cada uno.

In [ ]:
y_pred = categoricalNB.predict(X_train)
print(classification_report(y_train, y_pred, target_names=['No vuelve','Vuelve']))

EXPLICACION?????

In [ ]:
y_pred = categoricalNB.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

EXPLICACION???????

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [ ]:
fig, ax = plt.subplots(dpi=100)
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, vmin = 0, yticklabels=["No volveria","Volveria"], xticklabels=[ "No Volveria","Volveria"], ax=ax)
ax.set_title("Matriz de confusion")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

EXPLICACION???????

Graficamos ahora la curva ROC para ambos casos.

In [ ]:
graficarAUCROC("Categorical",categoricalNB,X_test,X_train,y_test,y_train)

EXPLICACION??????

In [ ]:
auc_cnb = roc_auc_score(y_test,categoricalNB.predict_proba(X_test)[:, 1])
print("AUC para categorical NB: {:.3f}".format(auc_cnb))

PREDICIR EL OTRO ARCHIVO Y GUARDAR LOS VALROES

In [ ]:
probabilidades_categorical = categoricalNB.predict_proba(X_train)
probabilidades_x_test_categorical = categoricalNB.predict_proba(X_test)

## MultinomialNB

In [ ]:
X_multinomialNB = multinomialNBPreprocessing(X)

In [ ]:
mejor_valor, mejor_alpha = obtenerMejorAlpha(MultinomialNB,X_multinomialNB)

Habiendo obtenido el mejor alpha para el Multinomial Naive Bayes, procedemos a desarrollarlo en mas detalle.

In [ ]:
print(mejor_alpha)
print(mejor_valor)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_multinomialNB, y, test_size=0.25, random_state=0)

In [ ]:
multinomialNB = MultinomialNB(alpha=mejor_alpha)

In [ ]:
multinomialNB.fit(X_train, y_train)

Realizamos ahora las predicciones correspondientes con la parte de entrenamiento y la de pruebas, una vez hecho eso, mostramos las metricas obtenidas en cada uno.

In [ ]:
y_pred_train = multinomialNB.predict(X_train)
print(classification_report(y_train, y_pred_train, target_names=['No vuelve','Vuelve']))

In [ ]:
y_pred_test = multinomialNB.predict(X_test)
print(classification_report(y_test, y_pred_test, target_names=['No vuelve','Vuelve']))

Como podemos observar, se obtuvieron en la mayoria de las metricas valores cercanos al 80% para ambas predicciones hechas. Vemos la matriz de confusion ahora y el area bajo la curva.

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

In [ ]:
fig, ax = plt.subplots(dpi=100)
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot = True, vmin = 0, yticklabels=["No volveria","Volveria"], xticklabels=[ "No Volveria","Volveria"], ax=ax)
ax.set_title("Matriz de confusion para MultinomialNB")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

EXPLICACION???????

Graficamos ahora la curva ROC para ambos casos.

In [ ]:
graficarAUCROC("Multinomial",multinomialNB,X_test,X_train,y_test,y_train)

EXPLICACION??????

In [ ]:
auc_mnb = roc_auc_score(y_test,y_pred_test)
print("AUC para multinomial NB: {:.3f}".format(auc_mnb))

In [ ]:
probabilidades_multinomial = multinomialNB.predict_proba(X_train)
probabilidades_x_test_multinomial = multinomialNB.predict_proba(X_test)

## GaussianNB

In [ ]:
X_gaussianNB = gaussianNBPreprocessing(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_gaussianNB, y, test_size=0.25, random_state=0)

In [ ]:
gaussianNB = GaussianNB()

In [ ]:
gaussianNB.fit(X_train, y_train)

Realizamos ahora las predicciones correspondientes con la parte de entrenamiento y la de pruebas, una vez hecho eso, mostramos las metricas obtenidas en cada uno.

In [ ]:
y_pred_train = gaussianNB.predict(X_train)
print(classification_report(y_train, y_pred_train, target_names=['No vuelve','Vuelve']))

In [ ]:
y_pred_test = gaussianNB.predict(X_test)
print(classification_report(y_test, y_pred_test, target_names=['No vuelve','Vuelve']))

Explicacion...


In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

In [ ]:
fig, ax = plt.subplots(dpi=100)
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot = True, vmin = 0, yticklabels=["No volveria","Volveria"], xticklabels=[ "No Volveria","Volveria"], ax=ax)
ax.set_title("Matriz de confusion para GaussianNB")
ax.set_xlabel("Predicho")
ax.set_ylabel("Real")

EXPLICACION???????

Graficamos ahora la curva ROC para ambos casos.

In [ ]:
graficarAUCROC("Gaussian",gaussianNB,X_test,X_train,y_test,y_train)

EXPLICACION??????

In [ ]:
auc_gnb = roc_auc_score(y_test,y_pred_test)
print("AUC para gaussian NB: {:.3f}".format(auc_gnb))

In [ ]:
probabilidades_gaussian = gaussianNB.predict_proba(X_train)
probabilidades_x_test_gaussian = gaussianNB.predict_proba(X_test)

## Ensamble de Naive Bayes

Viendo que sklearn tiene la limitacion de que no permite trabajar a la vez con variables categoricas y variables continuas, decidimos realizar un ensamble al que le pasamos las probabilidades de los 3 modelos hechos anteriormente. De esta forma, logramos obtener un modelo de Naive Bayes que trabaje con ambos tipos de variables.

In [ ]:
ensamble_gaussiano = GaussianNB()

In [ ]:
probabilidades = np.hstack((probabilidades_multinomial, probabilidades_categorical , probabilidades_gaussian))
probabilidades_x_test = np.hstack((probabilidades_x_test_multinomial, probabilidades_x_test_categorical , probabilidades_x_test_gaussian))

In [ ]:
ensamble_gaussiano.fit(probabilidades,y_train)

In [ ]:
y_pred = ensamble_gaussiano.predict(probabilidades_x_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

In [ ]:
auc_enb = roc_auc_score(y_test,y_pred)
print("AUC para el ensamble de NB: {:.3f}".format(auc_enb))

In [ ]:
Explicaciones...

## Conclusiones Naive Bayes